In [1]:
!pip install torch==2.0.1
!pip install flair

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.0.1-cp38-cp38-manylinux1_x86_64.whl (619.9 MB)
  Using cached triton-2.0.0-1-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled torch-2.1.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformer-smaller-training-vocab 0.3.2 requires torch!=2.0.1,<3.0.0,>=1.8.0, but you have torch 2.0.1 which is incompatible.

[notice] A new release of pip is ava

In [2]:
from flair.models import SequenceTagger
from flair.data import Sentence
import json
from tqdm import tqdm

/home/jovyan/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2023-10-21 09:24:14.218643: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-21 09:24:14.392725: I tensorflow/core/util/port.cc:104] oneDNN custom operations are 

In [3]:
ner_tagger = SequenceTagger.load("ner-ontonotes-fast")

def get_ner_over_percent(sentence_text, percent=0.1):
  # Create a Flair Sentence object
  sentence = Sentence(sentence_text)

  # Run NER tagging on the sentence
  ner_tagger.predict(sentence, return_probabilities_for_all_classes=True)

  tags = []

  for index, entity in enumerate(sentence):
      entity_tags = [
          (entity.text, x.value, str(x.score))
          for x in entity.tags_proba_dist["ner"]
          if x.score > percent and x.value != "O"
      ]
      entity_tags.sort(key=lambda x: x[2], reverse=True)
      if len(entity_tags) > 0:
          tags.append({index: entity_tags})

  return tags

2023-10-21 09:24:31,300 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC


In [4]:
pos_tagger = SequenceTagger.load("pos")

def get_pos(sentence_text):
    # Create a Flair Sentence object
    sentence = Sentence(sentence_text)

    # Run POS tagging on the sentence
    pos_tagger.predict(sentence)

    tags = []

    for index, entity in enumerate(sentence):
        tags.append({index: (entity.text, entity.tag)})

    return tags

2023-10-21 09:24:33,509 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [5]:
def store_taggings():
    # Open the input JSON file for reading
    input_file_path = "./squad_data_train.json"
    data = []

    with open(input_file_path) as f:
        for line in f:
            data.append(json.loads(line))

    #return data

    already_read_contexts = dict()

    # Iterate through each object in the JSON data and add the 'Test' property
    for i in tqdm(range(len(data))):
        obj = data[i]
        if obj['context'] not in already_read_contexts.keys():
            obj["NER_context"] = get_ner_over_percent(obj["context"], 0.1)
            obj["POS_context"] = get_pos(obj["context"])
            already_read_contexts[obj["context"]] = (obj["NER_context"], obj["POS_context"])
        else:
            obj["NER_context"] = already_read_contexts[obj['context']][0]
            obj["POS_context"] = already_read_contexts[obj['context']][1]
            obj["NER_question"] = get_ner_over_percent(obj["question"], 0.1)
            obj["POS_question"] = get_pos(obj["question"])

    # Open a new JSON file for writing
    output_file_path = "./squad_data_train_pos_ner_18.json"
    with open(output_file_path, "w") as output_file:
        for data_line in data:
            output_file.write(json.dumps(data_line) + "\n")

    print(f"Data with 'Test' property added has been saved to {output_file_path}")

In [ ]:
store_taggings()

  6%|▌         | 5212/87599 [13:28<5:06:40,  4.48it/s] 